In [1]:
import os
import pandas as pd
from nltk.corpus import stopwords
import nltk
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from numpy import asarray
from numpy import savetxt
import pickle


class Main:     
    def show_current_time(self):
        ini = time.time()
        tz_SP = pytz.timezone('America/Sao_Paulo') 
        now = datetime.now(tz_SP)
        current_time = now.strftime("%d/%m/%Y %H:%M:%S")
        print("Current Time =", current_time)         

base_path = 'D:/'
#base_path = '/content/drive/My Drive/Colab Notebooks/'

print("Versão dopandas: " + pd.__version__)  
  

Versão dopandas: 1.0.5


In [2]:
# roda na linha de comando : python -m spacy download pt_core_news_lg
import spacy
nlp = spacy.load("pt_core_news_sm")
all_stopwords_spacy = nlp.Defaults.stop_words
nltk.download('stopwords')
stops = set(stopwords.words('portuguese') + list(punctuation) + list(all_stopwords_spacy)
           + list(['ão','õe','ões','art','fls','vol','fl','janeiro','fevereiro','março','abril','maio','junho','julho',
                 'agosto','setembro','outubro','novembro','dezembro','josé','jose','maria',
                   'ii','iii','iv','vi','vii','viii','xix','xi','xii','xiii','xiv','xv','xvi','xvii','xviii','xix',
                  'xx']))


os.chdir(base_path+'/dou')
df_original = pd.read_pickle("dataframe_original_balanceado.pkl")
max_features = 100000

corpus = df_original['texto']
y = df_original['target'].to_numpy()
y = y.astype('int')

pipe = Pipeline([('count', CountVectorizer(analyzer='word', ngram_range=(1, 2),max_df= 0.75,
                            max_features= max_features, stop_words=frozenset(stops),
                                           token_pattern=r'[a-zA-Z\u00C0-\u00FF]{3,100}')),
                  ('tfid', TfidfTransformer(norm= 'l2', use_idf= True))]).fit(corpus)
X = pipe.fit_transform(corpus).toarray()

#print(pipe['count'].transform(corpus))
#print(pipe['tfid'].idf_)
#print(pipe['count'].get_feature_names())
#print(pipe['count'].vocabulary_['presidente'])
#print(pipe['tfid'].idf_[pipe['count'].vocabulary_['unidade']])

#df_idf = pd.DataFrame()
#for key, value in pipe['count'].vocabulary_.items():
    #print(key,value)
    #print(pipe['tfid'].idf_[value]
#    df_idf[key] = [pipe['tfid'].idf_[value]]
#print(X[0:])
#print(df_idf.head)
#print(stops)
print(X.shape)
print(y.shape)
#df_idf.to_pickle("dataframe_idf_treina.pkl")
 
outfile = open('pipeline_treina.pkl','wb')
pickle.dump(pipe,outfile)
outfile.close()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\draus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(1166, 100000)
(1166,)


Tratando variável categórica "tipo" (OneHotEncoder)

In [3]:
hot_enc = OneHotEncoder(sparse=False,handle_unknown='ignore')
X_words = X
X_tipo = hot_enc.fit_transform(df_original[['_tipo']])
print(X.shape)
print(y.shape)
X = np.concatenate((X_words,X_tipo),axis=1)

print(X.shape)

# save to csv file
#savetxt('X.csv', X, delimiter=',')
#savetxt('y.csv', y, delimiter=',')
outfile = open('X.pkl','wb')
pickle.dump(X,outfile)
outfile.close()
outfile = open('y.pkl','wb')
pickle.dump(y,outfile)
outfile.close()
outfile = open('hot_encoder_treina.pkl','wb')
pickle.dump(hot_enc,outfile)
outfile.close()


(1166, 100000)
(1166,)
(1166, 100152)


In [4]:


os.chdir(base_path+'/dou')
df_original = pd.read_pickle("dataframe_original_desbalanceado.pkl")

corpus = df_original['texto']
y = df_original['target'].to_numpy()
y = y.astype('int')

X = pipe.transform(corpus).toarray()

#print(pipe['count'].transform(corpus).toarray())
#print(pipe['tfid'].idf_)
#print(X[0:])
#print(stops)
print(X.shape)
print(y.shape)




(644, 100000)
(644,)


Tratando variável categórica "tipo" (OneHotEncoder) - Classes desbalanceadas

In [5]:
X_words = X
X_tipo = hot_enc.fit_transform(df_original[['_tipo']])

print(X.shape)
print(X_tipo.shape)
print(y.shape)
X = np.concatenate((X_words,X_tipo),axis=1)

print(X.shape)

outfile = open('X_d.pkl','wb')
pickle.dump(X,outfile)
outfile.close()
outfile = open('y_d.pkl','wb')
pickle.dump(y,outfile)
outfile.close()
# save to csv file
#savetxt('X_d.csv', X, delimiter=',')
#savetxt('y_d.csv', y, delimiter=',')


(644, 100000)
(644, 152)
(644,)
(644, 100152)
